# Deep Learning Toolkit for Splunk - Notebook for TensorFlow 2.0

## Neural Network for Binary Classification
This notebook contains an example workflow how to work on custom containerized code that seamlessly interfaces with the Deep Learning Toolkit for Splunk. As an example we use a custom binary neural network classifier built on keras and tensorflow.

Note: By default every time you save this notebook the cells are exported into a python module which is then invoked by Splunk MLTK commands like <code> | fit ... | apply ... | summary </code>. Please read the Model Development Guide in the Deep Learning Toolkit app for more information.

## Stage 0 - import libraries
At stage 0 we define all imports necessary to run our subsequent code depending on various libraries.

In [1]:
# mltkc_import
# this definition exposes all python module imports that should be available in all subsequent commands
import json
import datetime
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras

# global constants
MODEL_DIRECTORY = "/srv/app/model/data/"

In [2]:
# THIS CELL IS NOT EXPORTED - free notebook cell for testing purposes
print("numpy version: " + np.__version__)
print("pandas version: " + pd.__version__)
print("TensorFlow version: " + tf.__version__)
print("Keras version: " + keras.__version__)

numpy version: 1.18.1
pandas version: 1.0.1
TensorFlow version: 2.1.0
Keras version: 2.2.4-tf


## Stage 1 - get a data sample from Splunk
In Splunk run a search to pipe a prepared dataset into this environment.

| inputlookup  titanic-train.csv
| fit Imputer Age strategy=mean | eval Age = round(Imputed_Age / 90,1)  | fields - Imputed_Age
| search Embarked = * 
| eval SibSp = case(SibSp=0,"S0", SibSp=1,"S1", SibSp=2,"S2", SibSp=3,"S3", SibSp=4,"S4", SibSp=5,"S5",1=1,"S_More")
| eval Parch = case(Parch=0,"P0", Parch=1,"P1", Parch=2,"P2", Parch=3,"P3", Parch=4,"P4", Parch=5,"P5",1=1,"P_More")
| eventstats max(Fare) as max_Fare
| eval Fare =  round(Fare/max_Fare,1) | eval Fare = "F_" + Fare, Age = "Age_" + Age  | fields - max_Fare
| rex field=Name "\,\s(?<title>\w+)"  | fields - Name 
| eval title = case(title="Mr","Mr", title="Miss", "Miss", title="Mrs","Mrs",title="Master","Mastar",1=1,"Other")
| rex field=Ticket "(?<TicketNo>\d+)$" | fields - Ticket
| bin TicketNo span=100000
| eval TicketNo = case(TicketNo="0-100000","No0", TicketNo="100000-200000", "No100000", TicketNo="200000-300000", "No200000",TicketNo="300000-400000","No300000",1=1,"NoOther")
| rex field=Cabin "(?<Cabin>\w)" 
| eval Cabin = case(Cabin="T","Cabin_Unknown",Cabin="C","Cabin_C", isnull(Cabin),"Cabin_Unknown",1=1,Cabin)
| eval {Pclass} = 1, {Sex} = 1, {Embarked} =1, {title}=1, {Cabin} = 1, {TicketNo}=1, {SibSp}=1,{Parch}=1, {Fare} =1, {Age} = 1
| fillnull
| fields - Pclass, Sex, Embarked, title, count, Cabin, TicketNo, Fare, SibSp, Parch,Age, PassengerId, "Age_0.9"
| fit MLTKContainer mode=stage algo=mytitanic3 Survived from * into app:mytitanic3_model

After you run this search your data set sample is available as a csv inside the container to develop your model. The name is taken from the into keyword ("my_model" in the example above) or set to "default" if no into keyword is present. This step is intended to work with a subset of your data to create your custom model.

In [3]:
# mltkc_stage
# this cell is not executed from MLTK and should only be used for staging data into the notebook environment
def stage(name):
    with open("data/"+name+".csv", 'r') as f:
        df = pd.read_csv(f)
    with open("data/"+name+".json", 'r') as f:
        param = json.load(f)
    return df, param

In [4]:
# THIS CELL IS NOT EXPORTED - free notebook cell for testing purposes
df, param = stage("mytitanic3_model")
print(df[0:1])
print(df.shape)
print(str(param))

   Survived  3  male  S  Mr  Cabin_Unknown  No0  S1  P0  F_0.0  ...  S2  S5  \
0         0  1     1  1   1              1    1   1   1      1  ...   0   0   

   F  P3  Age_0.8  S_More  P4  F_1.0  F_0.4  P_More  
0  0   0        0       0   0      0      0       0  

[1 rows x 57 columns]
(889, 57)
{'options': {'params': {'mode': 'stage', 'algo': 'mytitanic3'}, 'args': ['Survived', '*'], 'target_variable': ['Survived'], 'feature_variables': ['*'], 'model_name': 'mytitanic3_model', 'algo_name': 'MLTKContainer', 'mlspl_limits': {'handle_new_cat': 'default', 'max_distinct_cat_values': '100', 'max_distinct_cat_values_for_classifiers': '100', 'max_distinct_cat_values_for_scoring': '100', 'max_fit_time': '600', 'max_inputs': '100000', 'max_memory_usage_mb': '1000', 'max_model_size_mb': '15', 'max_score_time': '600', 'streaming_apply': 'false', 'use_sampling': 'true'}, 'kfold_cv': None}, 'feature_variables': ['3', 'male', 'S', 'Mr', 'Cabin_Unknown', 'No0', 'S1', 'P0', 'F_0.0', 'Age_0.2', '1',

In [5]:
df

,Survived,3,male,S,Mr,Cabin_Unknown,No0,S1,P0,F_0.0,...,S2,S5,F,P3,Age_0.8,S_More,P4,F_1.0,F_0.4,P_More
0,0,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,0,0,0,1,1,1,0,...,0,0,0,0,0,0,0,0,0,0
2,1,1,0,1,0,1,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0
3,1,0,0,1,0,0,0,1,1,0,...,0,0,0,0,0,0,0,0,0,0
4,0,1,1,1,1,1,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
884,0,0,1,1,0,1,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0
885,1,0,0,1,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
886,0,1,0,1,0,1,1,1,0,1,...,0,0,0,0,0,0,0,0,0,0
887,1,0,1,0,1,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0


## Stage 2 - create and initialize a model

In [6]:
# mltkc_init
# initialize the model
# params: data and parameters
# returns the model object which will be used as a reference to call fit, apply and summary subsequently
def init(df,param):            
    X = df[param['feature_variables']]
    print("FIT build model with input shape " + str(X.shape))
    input_shape = int(X.shape[1])
    model = keras.Sequential()
    model.add(keras.layers.Dense(input_shape*2, input_dim=input_shape, activation=tf.nn.relu,
                                kernel_initializer='random_uniform'))
    model.add(keras.layers.Dense(32, activation=tf.nn.relu))
    model.add(keras.layers.BatchNormalization())
    model.add(keras.layers.Dropout(0.4))
    model.add(keras.layers.Dense(32, activation=tf.nn.relu))
    model.add(keras.layers.BatchNormalization())
    model.add(keras.layers.Dropout(0.4))
    model.add(keras.layers.Dense(32, activation=tf.nn.relu))
    model.add(keras.layers.BatchNormalization())
    model.add(keras.layers.Dropout(0.4))
    model.add(keras.layers.Dense(32, activation=tf.nn.relu))
    model.add(keras.layers.BatchNormalization())
    model.add(keras.layers.Dropout(0.4))
    model.add(keras.layers.Dense(32, activation=tf.nn.relu))
    model.add(keras.layers.BatchNormalization())
    model.add(keras.layers.Dropout(0.4))
    model.add(keras.layers.Dense(32, activation=tf.nn.relu))
    model.add(keras.layers.Dense(32, activation=tf.nn.relu))
    model.add(keras.layers.Dense(1, activation=tf.nn.sigmoid))
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [7]:
# test mltkc_stage_create_model
model = init(df,param)
print(model.summary())

FIT build model with input shape (889, 56)
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 112)               6384      
_________________________________________________________________
dense_1 (Dense)              (None, 32)                3616      
_________________________________________________________________
batch_normalization (BatchNo (None, 32)                128       
_________________________________________________________________
dropout (Dropout)            (None, 32)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 32)                1056      
_________________________________________________________________
batch_normalization_1 (Batch (None, 32)                128       
_________________________________________________________________
dropout_1 (Dr

In [8]:
df.shape

(889, 57)

## Stage 3 - fit the model

In [16]:
# mltkc_stage_create_model_fit
# returns a fit info json object
def fit(model,df,param):
    returns = {}            
    X = df[param['feature_variables']]
    Y = df[param['target_variables']]
    
    X_train, X_val = np.vsplit(X, [int(X.shape[0] * 0.8)])
    Y_train, Y_val = np.vsplit(Y, [int(Y.shape[0] * 0.8)])
    
    model_epochs = 300
    model_batch_size = 10
    if 'options' in param:
        if 'params' in param['options']:
            if 'epochs' in param['options']['params']:
                model_epochs = int(param['options']['params']['epochs'])
            if 'batch_size' in param['options']['params']:
                model_batch_size = int(param['options']['params']['batch_size'])
    # connect model training to tensorboard
    log_dir="/srv/notebooks/logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
    tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1, embeddings_freq=1)
    
    # Early Stopping
    earlystopping_callback= tf.keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0, patience=20, verbose=0, mode='auto')
    
    # run the training
    returns['fit_history'] = model.fit(x=X_train,
                                       y=Y_train, 
                                       verbose=2, 
                                       epochs=model_epochs, 
                                       batch_size=model_batch_size, 
                                       validation_data=(X_val, Y_val),
                                       callbacks=[tensorboard_callback,earlystopping_callback])
    # memorize parameters
    returns['model_epochs'] = model_epochs
    returns['model_batch_size'] = model_batch_size
    returns['model_loss_acc'] = model.evaluate(x = X, y = Y)
    return returns

In [10]:
returns = fit(model,df,param)
print(returns['model_loss_acc'])

Train on 711 samples, validate on 178 samples
Epoch 1/300
711/711 - 2s - loss: 0.7014 - accuracy: 0.5443 - val_loss: 0.6760 - val_accuracy: 0.6461
Epoch 2/300
711/711 - 0s - loss: 0.6967 - accuracy: 0.5387 - val_loss: 0.6599 - val_accuracy: 0.6461
Epoch 3/300
711/711 - 0s - loss: 0.6791 - accuracy: 0.5992 - val_loss: 0.6539 - val_accuracy: 0.6461
Epoch 4/300
711/711 - 0s - loss: 0.6790 - accuracy: 0.5879 - val_loss: 0.6508 - val_accuracy: 0.6461
Epoch 5/300
711/711 - 0s - loss: 0.6636 - accuracy: 0.5992 - val_loss: 0.6266 - val_accuracy: 0.6461
Epoch 6/300
711/711 - 0s - loss: 0.6536 - accuracy: 0.6245 - val_loss: 0.6304 - val_accuracy: 0.6461
Epoch 7/300
711/711 - 0s - loss: 0.6387 - accuracy: 0.6596 - val_loss: 0.5941 - val_accuracy: 0.6461
Epoch 8/300
711/711 - 0s - loss: 0.6291 - accuracy: 0.6568 - val_loss: 0.5402 - val_accuracy: 0.7191
Epoch 9/300
711/711 - 0s - loss: 0.5928 - accuracy: 0.6906 - val_loss: 0.5222 - val_accuracy: 0.7528
Epoch 10/300
711/711 - 0s - loss: 0.6158 - ac

## Stage 4 - apply the model

In [11]:
# mltkc_stage_create_model_apply
def apply(model,df,param):
    X = df[param['feature_variables']]
    y_hat = model.predict(x = X, verbose=0)
    return y_hat

In [18]:
# test mltkc_stage_create_model_apply
y_hat = apply(model,df,param)
#print(y_hat)

## Stage 5 - save the model

In [13]:
# save model to name in expected convention "<algo_name>_<model_name>.h5"
def save(model,name):
    # save keras model to hdf5 file
    # https://www.tensorflow.org/beta/tutorials/keras/save_and_restore_models
    model.save(MODEL_DIRECTORY + name + ".h5")
    return model

## Stage 6 - load the model

In [14]:
# load model from name in expected convention "<algo_name>_<model_name>.h5"
def load(name):
    model = keras.models.load_model(MODEL_DIRECTORY + name + ".h5")
    return model

## Stage 7 - provide a summary of the model

In [15]:
# return model summary
def summary(model=None):
    returns = {"version": {"tensorflow": tf.__version__, "keras": keras.__version__} }
    if model is not None:
        # Save keras model summary to string:
        s = []
        model.summary(print_fn=lambda x: s.append(x+'\n'))
        returns["summary"] = ''.join(s)
    return returns

## End of Stages
All subsequent cells are not tagged and can be used for further freeform code

# Fit SPL

| inputlookup  titanic-train.csv
| fit Imputer Age strategy=mean | eval Age = round(Imputed_Age / 90,1)  | fields - Imputed_Age
| search Embarked = * 
| eval SibSp = case(SibSp=0,"S0", SibSp=1,"S1", SibSp=2,"S2", SibSp=3,"S3", SibSp=4,"S4", SibSp=5,"S5",1=1,"S_More")
| eval Parch = case(Parch=0,"P0", Parch=1,"P1", Parch=2,"P2", Parch=3,"P3", Parch=4,"P4", Parch=5,"P5",1=1,"P_More")
| eventstats max(Fare) as max_Fare
| eval Fare =  round(Fare/max_Fare,1) | eval Fare = "F_" + Fare, Age = "Age_" + Age  | fields - max_Fare
| rex field=Name "\,\s(?<title>\w+)"  | fields - Name 
| eval title = case(title="Mr","Mr", title="Miss", "Miss", title="Mrs","Mrs",title="Master","Mastar",1=1,"Other")
| rex field=Ticket "(?<TicketNo>\d+)$" | fields - Ticket
| bin TicketNo span=100000
| eval TicketNo = case(TicketNo="0-100000","No0", TicketNo="100000-200000", "No100000", TicketNo="200000-300000", "No200000",TicketNo="300000-400000","No300000",1=1,"NoOther")
| rex field=Cabin "(?<Cabin>\w)" 
| eval Cabin = case(Cabin="T","Cabin_Unknown",Cabin="C","Cabin_C", isnull(Cabin),"Cabin_Unknown",1=1,Cabin)
| eval {Pclass} = 1, {Sex} = 1, {Embarked} =1, {title}=1, {Cabin} = 1, {TicketNo}=1, {SibSp}=1,{Parch}=1, {Fare} =1, {Age} = 1
| fillnull
| fields - Pclass, Sex, Embarked, title, count, Cabin, TicketNo, Fare, SibSp, Parch,Age, PassengerId, "Age_0.9"
| fit MLTKContainer algo=mytitanic3 Survived from * into app:mytitanic3_model
| eval predict = if(predicted_Survived > 0.5 , 1,0) 
| score f1_score Survived against predict

# Apply SPL

| inputlookup  titanic-test.csv
| fit Imputer Age strategy=mean | eval Age = round(Imputed_Age / 90,1)  | fields - Imputed_Age
| search Embarked = * 
| eval SibSp = case(SibSp=0,"S0", SibSp=1,"S1", SibSp=2,"S2", SibSp=3,"S3", SibSp=4,"S4", SibSp=5,"S5",1=1,"S_More")
| eval Parch = case(Parch=0,"P0", Parch=1,"P1", Parch=2,"P2", Parch=3,"P3", Parch=4,"P4", Parch=5,"P5",1=1,"P_More")
| eventstats max(Fare) as max_Fare
| eval Fare =  round(Fare/max_Fare,1) | eval Fare = "F_" + Fare, Age = "Age_" + Age  | fields - max_Fare
| rex field=Name "\,\s(?<title>\w+)"  | fields - Name 
| eval title = case(title="Mr","Mr", title="Miss", "Miss", title="Mrs","Mrs",title="Master","Mastar",1=1,"Other")
| rex field=Ticket "(?<TicketNo>\d+)$" | fields - Ticket
| bin TicketNo span=100000
| eval TicketNo = case(TicketNo="0-100000","No0", TicketNo="100000-200000", "No100000", TicketNo="200000-300000", "No200000",TicketNo="300000-400000","No300000",1=1,"NoOther")
| rex field=Cabin "(?<Cabin>\w)" 
| eval Cabin = case(Cabin="T","Cabin_Unknown",Cabin="C","Cabin_C", isnull(Cabin),"Cabin_Unknown",1=1,Cabin)
| eval {Pclass} = 1, {Sex} = 1, {Embarked} =1, {title}=1, {Cabin} = 1, {TicketNo}=1, {SibSp}=1,{Parch}=1, {Fare} =1, {Age} = 1
| fillnull
| fields - Pclass, Sex, Embarked, title, count, Cabin, TicketNo, Fare, SibSp, Parch,Age
| apply app:mytitanic3_model 
| eval Survived = if(predicted_Survived > 0.5 , 1,0) 
| table PassengerId Survived